Globecom Tutorial - December 7, 2021
Tutorial 29: Machine Learning for MIMO Systems with Large Arrays
Nuria Gonzalez-Prelcic (NCSU),
Aldebaro Klautau (UFPA) and
Robert W. Heath Jr. (NCSU)

This script uses Raymobtime data (https://www.lasse.ufpa.br/raymobtime/) and is based on the code used for the UFPA Problem Statement in the 2020 ITU Artificial Intelligence/Machine Learning in 5G Challenge (http://ai5gchallenge.ufpa.br/).

Authors: Aldebaro Klautau, Ailton Oliveira, Arthur Nascimento, Diego Gomes, Jamelly Ferreira, Walter Frazao

In [1]:
'''
Train a deep NN for choosing top-K beams.
Note that you need to download the datasets and save them in the folder specified in this code. 
'''

#Import modules
import os
import csv
import argparse
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.models import model_from_json,Model
from tensorflow.keras.layers import Dense,concatenate
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta,Adam
from sklearn.model_selection import train_test_split
from ml4comm.model_handler import ModelHandler

d:\Programs\Anaconda3\envs\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\Programs\Anaconda3\envs\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\Programs\Anaconda3\envs\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\Programs\Anaconda3\envs\python36\lib\site-packages\tensorflow\python\f

In [2]:
#Here you can choose a combination of sets of parameters to be used as the neural network input
coord = False  #coordinates with the receiver position obtained from GPS
img = False  #RGB images obtained from cameras
lidar = True  #three-dimensional histogram obtained form LIDAR point cloud

#Based on your choices above, make sure the datasets below can be found in the specific folders.
#If running on Colab, you may use /content/
COORDINATES_INPUT_FILE = "./data/coord_input/coord_input.npz"
IMAGES_INPUT_FILE = "./data/image_input/img_input_20.npz"
LIDAR_INPUT_FILE = "./data/lidar_input/lidar_input.npz"
BEAM_OUTPUT_FILE = "./data/beam_output/beams_output.npz"

In [3]:
###############################################################################
# Support functions
###############################################################################

#For description about top-k, including the explanation on how they treat ties (which can be misleading
#if your classifier is outputting a lot of ties (e.g. all 0's will lead to high top-k)
#https://www.tensorflow.org/api_docs/python/tf/nn/in_top_k
#define top-k accuracy for different values of k
def top_10_accuracy(y_true,y_pred):
    return metrics.top_k_categorical_accuracy(y_true,y_pred,k=10)

def top_30_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=30)

def top_50_accuracy(y_true,y_pred):
    return metrics.top_k_categorical_accuracy(y_true,y_pred,k=50)

def top_100_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=100)

#converts the combined channel gains into the label vector y
#to be used for training the model. For instance, if a softmax
#activation is used in the DNN output, then make y sum up to 1.
def prepare_output_labels(original_gains):  
    method = 1 #for using DNN output with softmax activation    
    if method == 1:
        thresholdBelowMax = 60 #threshold in dB for zeroing gains in log domain
        y = log_normalize_channel_gain(original_gains,thresholdBelowMax)
    return y

#converts to log scale and make zero the values below the specified threshold
def log_normalize_channel_gain(y,thresholdBelowMax):
    num_gains = len(y)
    #print('AK',num_gains)
    for i in range(0,y.shape[0]):            
            thisOutputs = y[i,:]
            logOut = 20*np.log10(thisOutputs + 1e-30)
            minValue = np.amax(logOut) - thresholdBelowMax
            zeroedValueIndices = logOut < minValue
            thisOutputs[zeroedValueIndices]=0
            #normalize to sum up to 1, and enable using softmax activation
            thisOutputs = thisOutputs / sum(thisOutputs)
            y[i,:] = thisOutputs        
    return y

#instead of representing the pair of beam indices with two integers, use only one integer.
#for instance, if yMatrix has a shape of (11194, 8, 32), where 8 is the number of Rx antennas
#and 32 the number of Tx antennas, then y_output has the shape (11194, 256) because 256=8*32.
#this way, a single index will indicate the pair of indices
def convert_pair_to_index(yMatrix):
    yMatrix = np.abs(yMatrix)
    yMatrix /= np.max(yMatrix)
    yMatrixShape = yMatrix.shape
    num_classes = yMatrix.shape[1] * yMatrix.shape[2]
    y = yMatrix.reshape(yMatrix.shape[0],num_classes)
    return y, num_classes

In [4]:
###############################################################################
# Data configuration
###############################################################################
tf.device('/device:GPU:0') #in case you have a GPU

num_epochs = 2 #assume a rather small number just to demo things. You should try larger values
batch_size = 32 #mini-batch size
validation_fraction = 0.1 #fraction of all examples that will compose the validation set
test_fraction = 0.2 #fraction of all examples that will compose the test set

seed = 7 #use a fixed number if you want to reproduce the same results
np.random.seed(seed)

In [5]:
#read the datasets with input parameters
if coord == True: 
    print("Reading dataset... ",COORDINATES_INPUT_FILE)
    coord_train_cache_file = np.load(COORDINATES_INPUT_FILE)
    X_coord = coord_train_cache_file['coordinates']    

In [6]:
if img == True:
    resizeFac = 20 # Resize Factor
    nCh = 1 # The number of channels of the image
    imgDim = (360,640) # Image dimensions in number of pixels
    #method = 1
    print("Reading dataset... ",IMAGES_INPUT_FILE)
    img_train_cache_file = np.load(IMAGES_INPUT_FILE)
    X_img = img_train_cache_file['inputs']

In [7]:
if lidar == True:
    print("Reading dataset... ",LIDAR_INPUT_FILE)
    lidar_train_cache_file = np.load(LIDAR_INPUT_FILE)
    X_lidar = lidar_train_cache_file['input']

Reading dataset...  ./data/lidar_input/lidar_input.npz


In [8]:
#read the dataset with output parameters    
print("Reading dataset...", BEAM_OUTPUT_FILE)
output_cache_file = np.load(BEAM_OUTPUT_FILE)
yMatrix = output_cache_file['output_classification']
y_output, num_classes = convert_pair_to_index(yMatrix)
y = prepare_output_labels(y_output)

#y_train, y_validation = train_test_split(y_output, test_size=0.2, random_state=seed, shuffle=True)

Reading dataset... ./data/beam_output/beams_output.npz


In [9]:
#compose input and output, and also the three sets (train, test, validation)
multimodal = [coord, img, lidar]
num_features_categories = sum(multimodal) 
if num_features_categories == 1:
    #single set of parameters
    if coord:
        X = X_coord
    elif img:
        X = X_img
    elif lidar:
        X = X_lidar
elif num_features_categories == 3:
    X = (X_coord, X_img, X_lidar)
elif num_features_categories == 2:
    if coord and lidar:
        X = (X_coord, X_lidar)
    elif coord and img:
        X = (X_coord, X_img)
    elif img and lidar:
        X = (X_img, X_lidar)

In [10]:
if num_features_categories == 1:
    num_examples = X.shape[0]
else:
    num_examples = X[0].shape[0]
print('num_examples=', num_examples)

num_examples= 11194


In [11]:
#split examples in 3 sets: train, test, validation
#use the indices of the examples
indices = np.arange(num_examples)
indices_train, indices_test_and_val = train_test_split(indices, test_size=validation_fraction+test_fraction,
                                                       random_state=seed, shuffle=True)
indices_validation, indices_test = train_test_split(indices_test_and_val, test_size=test_fraction/(validation_fraction+test_fraction),
                                                       shuffle=True)


In [12]:
#creates the 3 disjoint sets for the outputs
y_train = y[indices_train]
y_test = y[indices_test]
y_validation = y[indices_validation]

In [13]:
#creates the 3 disjoint sets for the inputs
if num_features_categories == 1:
    X_train = X[indices_train]
    X_test = X[indices_test]
    X_validation = X[indices_validation]
elif num_features_categories == 2:
    X_train = (X[0][indices_train], X[1][indices_train])
    X_test = (X[0][indices_test], X[1][indices_test])
    X_validation = (X[0][indices_validation], X[1][indices_validation])    
elif num_features_categories == 3:
    X_train = (X[0][indices_train], X[1][indices_train], X[2][indices_train])
    X_test = (X[0][indices_test], X[1][indices_test], X[2][indices_test])
    X_validation = (X[0][indices_validation], X[1][indices_validation], X[2][indices_validation])    

In [ ]:
##############################################################################
# Model configuration
##############################################################################
plot = True # Active Plot output

modelHandler = ModelHandler() #this class create some DNN models
opt = Adam() #optimizer

if coord:
    coord_model = modelHandler.createArchitecture('coord_mlp',num_classes,X_coord.shape[1],'complete')
if img:
    if nCh==1:   
        img_model = modelHandler.createArchitecture('light_image',num_classes,[X_img.shape[1],X_img.shape[2],1],'complete')
    else:
        img_model = modelHandler.createArchitecture('light_image',num_classes,[X_img.shape[1],X_img.shape[2],X_img.shape[3]],'complete')
if lidar:
    lidar_model = modelHandler.createArchitecture('lidar_simple',num_classes,[X_lidar.shape[1],X_lidar.shape[2],X_lidar.shape[3]],'complete')

if num_features_categories == 2:
    #recall order: (X_coord, X_img, X_lidar)
    if coord and lidar:
        combined_model = concatenate([coord_model.output,lidar_model.output])
        z = Dense(num_classes,activation="relu")(combined_model)
        model = Model(inputs=[coord_model.input,lidar_model.input],outputs=z)
    elif coord and img:
        combined_model = concatenate([coord_model.output,img_model.output])
        z = Dense(num_classes,activation="relu")(combined_model)
        model = Model(inputs=[coord_model.input,img_model.input],outputs=z)    
    else:
        combined_model = concatenate([img_model.output, lidar_model.output])
        z = Dense(num_classes,activation="relu")(combined_model)
        model = Model(inputs=[img_model.input, lidar_model.input],outputs=z)

elif num_features_categories == 3:
    combined_model = concatenate([coord_model.output,img_model.output,lidar_model.output])
    z = Dense(num_classes,activation="relu")(combined_model)
    model = Model(inputs=[coord_model.input,img_model.input,lidar_model.input],outputs=z)
else:
    if coord:
        model = coord_model
    elif img:
        model = img_model  
    else:
        model = lidar_model
        
model.compile(loss=categorical_crossentropy,
                    optimizer=opt,
                    metrics=[metrics.categorical_accuracy,
                            metrics.top_k_categorical_accuracy,
                            top_10_accuracy,
                            top_30_accuracy,
                            top_50_accuracy,
                            top_100_accuracy])

print(model.summary())

#train stage
#note that Keras will properly deal with num_features_categories=1, 2 or 3.
#for num_features_categories > 1, Keras will detect it is a tuple and feed
#the distinct neural network inputs with the correct data
hist = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size,
                     validation_data=(X_validation, y_validation))

#save training statistics
with open('history.txt', 'w') as f: 
       f.write(str(hist.history))

if plot:
       k_beams = [1, 5, 10, 30, 50]
       fig, ax = plt.subplots(figsize=(7,4))
       # acurracy's
       y = [max(hist.history['categorical_accuracy']),
       max(hist.history['top_k_categorical_accuracy']),
       max(hist.history['top_10_accuracy']),
       max(hist.history['top_30_accuracy']),
       max(hist.history['top_50_accuracy'])]
       ax.plot(k_beams,y, 'r--s', label = 'Beam selection accuracy')
       # original labels
       '''ax.plot(k_beams,y_ori, 'b--s', label = 'Correct orientation-LIDAR')
       ax.plot(k_beams,y_sori, 'k--s', label = 'Fixed oientation-LIDAR ')
       ax.plot(k_beams,y_MM_ori, 'm--s', label = 'Correct orientation-MM')
       ax.plot(k_beams,y_MM_sori, 'r--s', label = 'Fixed oientation-MM')'''
       ax.set(xlabel='Top K', ylabel='Accuracy')
       plt.xlim(right=51)
       ax.grid()
       plt.legend()
       plt.show()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 200, 10)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 200, 10)       16910     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 200, 10)       12110     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 40, 10)         0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 40, 10)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 40, 10)         4910      
_________

## **Accuracy per epochs on training set**

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(hist.history['categorical_accuracy'], 'b', label = 'Categorical accuracy', linewidth=2)
ax.plot(hist.history['top_k_categorical_accuracy'], 'k--', label = 'Top 5', linewidth=2)
ax.plot(hist.history['top_10_accuracy'], 'm', label = 'Top 10', linewidth=2)
ax.plot(hist.history['top_30_accuracy'], 'g--', label = 'Top 30', linewidth=2)
ax.plot(hist.history['top_50_accuracy'], 'r', label = 'Top 50', linewidth=2)
ax.set(xlabel='Epochs', ylabel='Accuracy')
ax.grid()
plt.legend()
plt.show()

## **Accuracy for k=1 on test set**

In [ ]:
#test stage
model_predict = model.predict(X_test)
y_predicted_best_index = np.argmax(model_predict, axis=1)
y_correct_best_index = np.argmax(y_test, axis=1)
from sklearn.metrics import accuracy_score
print('top-1 accuracy=', accuracy_score(y_correct_best_index, y_predicted_best_index))